In [ ]:
import rp
import source.peekaboo as peekaboo
from source.peekaboo import run_peekaboo
from source.clip import get_clip_logits
from source.stable_diffusion import StableDiffusion
from IPython.display import clear_output

In [ ]:
device=rp.select_torch_device()
StableDiffusion(device=device);

In [ ]:
#Mario vs Luigi Part 1
results_collection=[]

In [ ]:
def get_clip_logits_per_image(prompt, images):
    return [get_clip_logits(image, [prompt])[0] for image in images]

def random_colors(length=100):
    return [rp.random_rgb_float_color() for _ in range(length)]

def get_score(foreground, alpha, prompt:str, colors:list):
    
    alpha=alpha>.5
    rp.display_image(rp.blend_images(rp.random_rgb_float_color(),rp.cv_resize_image(foreground,rp.get_image_dimensions(alpha)),alpha))
    alpha=rp.cv_dilate(alpha,5,circular=True)
    
    assert rp.is_image(foreground) and rp.is_image(alpha)
    images = [rp.blend_images(foreground, color, alpha) for color in colors]
    scores = get_clip_logits_per_image(prompt, images)
    score = rp.mean(scores)
    return score

In [ ]:
def ranked_results(results):
    colors = random_colors()
    def score(result):
        output = get_score(foreground = result.image,
                         alpha = result.alphas[0],
                         prompt = result.p_name,
                         colors = colors,
                        )
        print(output)
        return output
    scores = list(map(score,results))
    scores, results = rp.sync_sort(scores, results)
    
    #First is best
    scores, results = scores[::-1], results[::-1]
    
    return scores, results

def display_ranked_results(scores, results):
    for score, result in zip(scores, results):
        rp.display_image(
            rp.labeled_image(
                rp.horizontally_concatenated_images(
                    rp.cv_resize_image(result.image, (256, 256)), result.alphas[0]
                ),
                "%s : %f" % (result.p_name, score),
            )
        )

In [ ]:
thumbnails=[]
def get_result_thumbnail(results):
    alpha, image = results.alphas[0], results.image
    height = width = 512 
    image = rp.cv_resize_image(image, (height, width))
    alpha = rp.cv_resize_image(alpha, (height, width),interp='nearest')
    method = " + ".join(
        [
            *(["CLIP"] if results.clip_coef else []),
            *(["StableDreamLoss"] if results.use_stable_dream_loss else []),
        ]
    )
    path = rp.get_relative_path(results.output_folder)
    name = results.p_name
    settings=[
        'representation',
        'LEARNING_RATE',
        'NUM_ITER',
        'GRAVITY',
        'clip_coef',
        'use_stable_dream_loss',
        'GUIDANCE_SCALE',
        'min_step',
        'max_step',
        'clip_coef',
    ]
    settings=[x+': '+str(results[x]) for x in settings]
    
    
    text=rp.line_join([name, ' ', method, ' ', path, '', 'Settings:',*settings])
    text=rp.wrap_string_to_width(text,60)
    text = rp.cv_text_to_image(text,monospace=False)
    text = rp.resize_image_to_fit(text, height, width)
    image=rp.horizontally_concatenated_images(image,alpha,text)
    
    out_dir='thumbnails_data6'
    rp.make_directory(out_dir)
    out_name='%s ____ %s ____ %i.png'%(method, name,len(rp.get_all_files(out_dir)))
    out_path=rp.path_join(out_dir,out_name)
    rp.save_image(image,out_path)
    
    thumbnails.append(image)
    rp.display_image(image)
    return image

In [ ]:
experiment_input_data=[       [ [
            # " Harry Potter boy - daniel radcliffe - man with glasses",
            "Hermoine Granger - Emma Watson - Girl with long hair",
            # "Hermoine",
            # "Harry Potter",
            # "Daniel Radcliffe",
            # "Emma Watson",
        ],
        [
            "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
            # "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
        ],]]

In [ ]:

import icecream
import numpy as np
import rp
import torch
import torch.nn as nn
from easydict import EasyDict
from IPython.display import clear_output
from torchvision.transforms.functional import normalize

import source.stable_diffusion as sd
from source.bilateral_blur import BilateralProxyBlur
from source.learnable_textures import (LearnableImageFourier,
                                       LearnableImageFourierBilateral,
                                       LearnableImageRaster,
                                       LearnableImageRasterBilateral,
                                       LearnableTexturePackFourier,
                                       LearnableTexturePackRaster,
                                      LearnableImageRasterSigmoided)

from typing import Union, List, Optional
import rp
from easydict import EasyDict
from collections import defaultdict
import pandas as pd
from icecream import ic

from source.stable_diffusion_labels import get_mean_embedding, BaseLabel, SimpleLabel, MeanLabel

def make_learnable_image(height, width, num_channels, foreground=None, bilateral_kwargs:dict={}, representation = 'fourier'):
    #Here we determine our image parametrization schema
    bilateral_blur =  BilateralProxyBlur(foreground,**bilateral_kwargs)
    if representation=='fourier bilateral':
        return LearnableImageFourierBilateral(bilateral_blur,num_channels) #A neural neural image + bilateral filter
    elif representation=='raster bilateral':
        return LearnableImageRasterBilateral(bilateral_blur,num_channels) #A regular image + bilateral filter
    elif representation=='fourier':
        return LearnableImageFourier(height,width,num_channels) #A neural neural image
    elif representation=='raster':
        return LearnableImageRasterSigmoided(height,width,num_channels) #A regular image
    else:
        assert False, 'Invalid method: '+representation

def blend_torch_images(foreground, background, alpha):
    #Input assertions
    assert foreground.shape==background.shape
    C,H,W=foreground.shape
    assert alpha.shape==(H,W), 'alpha is a matrix'
    
    return foreground*alpha + background*(1-alpha)

class PeekabooSegmenter(nn.Module):
    def __init__(self,
                 image:np.ndarray,
                 labels:List['BaseLabel'],
                 size:int=256,
                 name:str='Untitled',
                 bilateral_kwargs:dict={},
                 representation = 'fourier bilateral',
                 min_step=None,
                 max_step=None,
                ):
        
        s=sd._get_stable_diffusion_singleton()
        
        super().__init__()
        
        height=width=size #We use square images for now
        
        assert all(issubclass(type(label),BaseLabel) for label in labels)
        assert len(labels), 'Must have at least one class to segment'
        
        self.height=height
        self.width=width
        self.labels=labels
        self.name=name
        self.representation=representation
        self.min_step=s.min_step if min_step is None else min_step
        self.max_step=s.max_step if max_step is None else max_step
        
        assert rp.is_image(image), 'Input should be a numpy image'
        image=rp.cv_resize_image(image,(height,width))
        image=rp.as_rgb_image(image) #Make sure it has 3 channels in HWC form
        image=rp.as_float_image(image) #Make sure it's values are between 0 and 1
        assert image.shape==(height,width,3) and image.min()>=0 and image.max()<=1
        self.image=image
        
        self.foreground=rp.as_torch_image(image).to(s.device) #Convert the image to a torch tensor in CHW form
        assert self.foreground.shape==(3, height, width)
        
        self.background=self.foreground*0 #The background will be a solid color for now
        
        self.alphas=make_learnable_image(height,width,num_channels=self.num_labels,foreground=self.foreground,representation=self.representation,bilateral_kwargs=bilateral_kwargs)
            
    @property
    def num_labels(self):
        return len(self.labels)
            
    def set_background_color(self, color):
        r,g,b = color
        assert 0<=r<=1 and 0<=g<=1 and 0<=b<=1
        self.background[0]=r
        self.background[1]=g
        self.background[2]=b
        
    def randomize_background(self):
        self.set_background_color(rp.random_rgb_float_color())
        
    def forward(self, alphas=None, return_alphas=False):        
        s=sd._get_stable_diffusion_singleton()
        
        try:
            old_min_step=s.min_step
            old_max_step=s.max_step
            s.min_step=self.min_step
            s.max_step=self.max_step

            output_images = []

            if alphas is None:
                alphas=self.alphas()

            assert alphas.shape==(self.num_labels, self.height, self.width)
            assert alphas.min()>=0 and alphas.max()<=1

            for alpha in alphas:
                output_image=blend_torch_images(foreground=self.foreground, background=self.background, alpha=alpha)
                output_images.append(output_image)

            output_images=torch.stack(output_images)

            assert output_images.shape==(self.num_labels, 3, self.height, self.width) #In BCHW form

            if return_alphas:
                return output_images, alphas
            else:
                return output_images

        finally:
            old_min_step=s.min_step
            old_max_step=s.max_step 

def display(self):
    #This is a method of PeekabooSegmenter, but can be changed without rewriting the class if you want to change the display

    colors = [(1,1,1), (0,0,0), (1,0,0),(0,1,0),(0,0,1)]#(1,0,0), (0,1,0), (0,0,1)] #Colors used to make the display
    # colors = [rp.random_rgb_float_color() for _ in range(3)]
    with torch.no_grad():
        alphas = rp.as_numpy_array(self.alphas())
    image = self.image
    assert alphas.shape==(self.num_labels, self.height, self.width)

    composites = []
    for color in colors:
        self.set_background_color(color)
        column=rp.as_numpy_images(self(self.alphas()))
        composites.append(column)

    label_names=[label.name for label in self.labels]

    stats_lines = [
        self.name,
        '',
        'H,W = %ix%i'%(self.height,self.width),
    ]

    def try_add_stat(stat_format, var_name):
        if var_name in globals():
            stats_line=stat_format%globals()[var_name]
            stats_lines.append(stats_line)

    try_add_stat('Gravity: %.2e','GRAVITY'   )
    try_add_stat('Batch Size: %i','BATCH_SIZE')
    try_add_stat('Iter: %i','iter_num')
    try_add_stat('Image Name: %s','image_filename')
    try_add_stat('Learning Rate: %.2e','LEARNING_RATE')
    try_add_stat('Guidance: %i%%','GUIDANCE_SCALE')

    stats_image=rp.labeled_image(self.image, rp.line_join(stats_lines), 
                                 size=15*len(stats_lines), 
                                 position='bottom', align='center')

    composite_grid=rp.grid_concatenated_images([
        rp.labeled_images(alphas,label_names),
        *composites
    ])
    
    assert rp.is_image(self.image)
    assert rp.is_image(alphas[0])
    assert rp.is_image(composites[0][0])
    assert rp.is_image(composites[1][0])
    assert rp.is_image(composites[2][0])

    output_image = rp.labeled_image(
        rp.tiled_images(
            rp.labeled_images(
                [
                    self.image,
                    alphas[0],

                    composites[0][0],
                    composites[1][0],
                    composites[2][0],
                    composites[3][0],
                    composites[4][0],
                ],
                [
                    "Input Image",
                    "Alpha Map",
                    "Background #1",
                    "Background #2",
                    "Background #3",
                    "Background #4",
                    "Background #5",
                ],
            ),
            length=2 + len(composites),
            border_thickness=0,
        ),
        label_names[0],
    )


    # output_image = rp.horizontally_concatenated_images(stats_image, composite_grid)

    rp.display_image(output_image)

    return output_image

PeekabooSegmenter.display=display
    
def log_cell(cell_title):
    rp.fansi_print("<Cell: %s>"%cell_title, 'cyan', 'underlined')
    # rp.ptoc()
def log(x):
    x=str(x)
    rp.fansi_print(x, 'yellow')

class PeekabooResults(EasyDict):
    #Acts like a dict, except you can read/write parameters by doing self.thing instead of self['thing']
    pass

def save_peekaboo_results(results,new_folder_path):
    assert not rp.folder_exists(new_folder_path), 'Please use a different name, not %s'%new_folder_path
    rp.make_folder(new_folder_path)
    with rp.SetCurrentDirectoryTemporarily(new_folder_path):
        log("Saving PeekabooResults to "+new_folder_path)
        params={}
        rp.save_json(params,'params.json',pretty=True)
        for key in results:
            value=results[key]
            if rp.is_image(value): 
                #Save a single image
                rp.save_image(value,key+'.png')
            elif isinstance(value, np.ndarray) and rp.is_image(value[0]):
                #Save a folder of images
                rp.make_directory(key)
                with rp.SetCurrentDirectoryTemporarily(key):
                    
                    
                    names=[]
                    for i in range(len(value)):
                        name='%03i.png'%i
                        names.append(name)
                        # rp.save_image(value[i],str(i)+'.png')
                    rp.save_images(value,names,show_progress=True)
            elif isinstance(value, np.ndarray):
                #Save a generic numpy array
                np.save(key+'.npy',value) 
            else:

                import json
                try:
                    json.dumps({key:value})
                    #Assume value is json-parseable
                    params[key]=value
                except Exception:
                    params[key]=str(value)
        log("Done saving PeekabooResults to "+new_folder_path+"!")
        
        
class PeekabooResult:
    def __init__(self,root):
        self.root=root

    def sub(self,path):
        return rp.path_join(self.root,path)

    def img(self,path):
        return rp.as_float_image(rp.load_image(self.sub(path),use_cache=True))

    @property
    def is_valid(self):
        try:
            #Are all the associated paths there?
            self.params
            return True
        except Exception:
            return False
    
    @rp.memoized_property
    def image_path(self):
        return self.params.image_path

    @rp.memoized_property
    def image_name(self):
        return rp.get_file_name(self.image_path)

    @rp.memoized_property
    def name(self):
        return self.params.p_name

    @rp.memoized_property
    def params(self):
        return EasyDict(rp.load_json(self.sub('params.json')))

    @rp.memoized_property
    def image(self):
        return rp.as_rgb_image(rp.as_float_image(self.img('image.png')))
    
    @rp.memoized_property
    def scaled_image(self):
        return rp.cv_resize_image(self.image,rp.get_image_file_dimensions(self.alpha_path))

    @rp.memoized_property
    def alpha(self):
        return rp.as_grayscale_image(rp.as_float_image(self.img('alphas/0.png')))
    
    @rp.memoized_property
    def alpha_path(self):
        return self.sub('alphas/0.png')

    @rp.memoized_property
    def preview_image(self):
        return self.img('preview_image.png')

    def __repr__(self):
        return 'PeekabooResult(%s)'%(self.name)
        
def make_image_square(image:np.ndarray, method='crop')->np.ndarray:
    #Takes any image and makes it into a 512x512 square image with shape (512,512,3)
    assert rp.is_image(image)
    assert method in ['crop','scale']
    image=rp.as_rgb_image(image)
    
    height, width = rp.get_image_dimensions(image)
    min_dim=min(height,width)
    max_dim=max(height,width)
    
    if method=='crop':
        return make_image_square(rp.crop_image(image, min_dim, min_dim, origin='center'),'scale')
    if method=='scale':
        return rp.resize_image(image, (512,512))
                    

def run_peekaboo(name:str, image:Union[str,np.ndarray], label:Optional['BaseLabel']=None,
                
                #Peekaboo Hyperparameters:
                GRAVITY=1e-1/2, # This is the one that needs the most tuning, depending on the prompt...
                #   ...usually one of the following GRAVITY will work well: 1e-2, 1e-1/2, 1e-1, or 1.5*1e-1
                NUM_ITER=300,       # 300 is usually enough
                LEARNING_RATE=1e-5, # Can be larger if not using neural neural textures (aka when representation is raster)
                BATCH_SIZE=1,       # Doesn't make much difference, larger takes more vram
                GUIDANCE_SCALE=100, # The defauly value from the DreamFusion paper
                bilateral_kwargs=dict(kernel_size = 3,
                                      tolerance = .08,
                                      sigma = 5,
                                      iterations=40,
                                     ),
                square_image_method='crop', #Can be either 'crop' or 'scale' - how will we square the input image?
                representation='fourier bilateral', #Can be 'fourier bilateral', 'raster bilateral', 'fourier', or 'raster'
                min_step=None,
                max_step=None,
                clip_coef=0,
                use_stable_dream_loss=True,
                 output_folder_name='peekaboo_results',
                )->PeekabooResults:
    
    s=sd._get_stable_diffusion_singleton()
    
    if label is None: 
        label=SimpleLabel(name)
    
    image_path='<No image path given>'
    if isinstance(image,str):
        image_path=image
        image=rp.load_image(image)
    
    assert rp.is_image(image)
    assert issubclass(type(label),BaseLabel)
    image=rp.as_rgb_image(rp.as_float_image(make_image_square(image,square_image_method)))
    rp.tic()
    time_started=rp.get_current_date()
    
    
    log_cell('Get Hyperparameters') ########################################################################
    icecream.ic(GRAVITY, BATCH_SIZE, NUM_ITER, LEARNING_RATE, GUIDANCE_SCALE,  representation, bilateral_kwargs, square_image_method)



    # log_cell('Alpha Initializer') ########################################################################

    p=PeekabooSegmenter(image,
                        labels=[label],
                        name=name,
                        bilateral_kwargs=bilateral_kwargs,
                        representation=representation, 
                        min_step=min_step,
                        max_step=max_step,
                       ).to(s.device)

    if 'bilateral' in representation:
        blur_image=rp.as_numpy_image(p.alphas.bilateral_blur(p.foreground))
        print("The bilateral blur applied to the input image before/after, to visualize it")
        rp.display_image(rp.tiled_images(rp.labeled_images([rp.as_numpy_image(p.foreground),blur_image],['before','after'])))

    p.display();


    
    # log_cell('Create Optimizers') ########################################################################

    params=list(p.parameters())
    optim=torch.optim.Adam(params,lr=1e-3)
    optim=torch.optim.SGD(params,lr=LEARNING_RATE)


    # log_cell('Create Logs') ########################################################################
    global iter_num
    iter_num=0
    timelapse_frames=[]


    # log_cell('Do Training') ########################################################################
    preview_interval=NUM_ITER//10 #Show 10 preview images throughout training to prevent output from being truncated
    preview_interval=1 #I want EVERY FRAME!!!
    preview_interval=max(1,preview_interval)
    log("Will show preview images every %i iterations"%(preview_interval))

    try:
        display_eta=rp.eta(NUM_ITER)
        for _ in range(NUM_ITER):
            display_eta(_)
            iter_num+=1

            alphas=p.alphas()

            for __ in range(BATCH_SIZE):
                p.randomize_background()
                composites=p()
                for label, composite in zip(p.labels, composites):
                    if clip_coef>0: 
                        #Use clip instead of stable-dream-loss
                        #You must use 'name' for the prompt in this case
                        from .clip import get_clip_logits
                        logit=get_clip_logits(composite, label.name)*clip_coef
                        loss=-logit
                        loss.sum().backward(retain_graph=True)
                        print(float(loss.sum()))
                    if use_stable_dream_loss:
                        s.train_step(label.embedding, composite[None], 
                                     guidance_scale=GUIDANCE_SCALE
                                    )
                        

            ((alphas.sum())*GRAVITY).backward(retain_graph=True)

            optim.step()
            optim.zero_grad()

            with torch.no_grad():
                if not _%100:
                    # Don't overflow the notebook
                    clear_output()
                if not _%preview_interval: 
                    timelapse_frames.append(p.display())
                    # rp.ptoc()
    except KeyboardInterrupt:
        log("Interrupted early, returning current results...")
        pass

    output_folder = rp.make_folder('%s/%s'%(output_folder_name,name))
    output_folder += '/%03i'%len(rp.get_subfolders(output_folder))
    
                
    # rp.ptoc()
    results = PeekabooResults(
        #The main output is the alphas
        alphas=rp.as_numpy_array(alphas),
        
        #Keep track of hyperparameters used
        GRAVITY=GRAVITY,
        BATCH_SIZE=BATCH_SIZE,
        NUM_ITER=NUM_ITER,
        GUIDANCE_SCALE=GUIDANCE_SCALE,
        LEARNING_RATE=LEARNING_RATE,
        bilateral_kwargs=bilateral_kwargs,
        representation=representation,
        
        #Keep track of the inputs used
        label=label,
        image=image,
        image_path=image_path,
        clip_coef=clip_coef,
        
        use_stable_dream_loss=use_stable_dream_loss,
        #Record some extra info
        preview_image=p.display(),
        timelapse_frames=rp.as_numpy_array(timelapse_frames),
        **({'blur_image':blur_image} if 'blur_image' in dir() else {}),
        height=p.height,
        width=p.width,
        p_name=p.name,
        output_folder=rp.get_absolute_path(output_folder),
        
        min_step=p.min_step,
        max_step=p.max_step,
        
        git_hash=rp.get_current_git_hash(), 
        time_started=rp.r._format_datetime(time_started),
        time_completed=rp.r._format_datetime(rp.get_current_date()),
        device=s.device,
        computer_name=rp.get_computer_name(),
    ) 

    save_peekaboo_results(results,output_folder)
    print("Please wait - creating a training timelapse")
    
    clear_output()
    rp.display_image_slideshow(timelapse_frames)#This can take a bit of time
    print("Saved results at %s"%output_folder)
    icecream.ic(name,label,image_path, GRAVITY, BATCH_SIZE, NUM_ITER, GUIDANCE_SCALE,  bilateral_kwargs)
    
    return results


In [ ]:
experiment_setting_presets = [
    # dict(
    #     representation="raster bilateral",
    #     LEARNING_RATE=1e-0,
    #     # NUM_ITER=500,
    #     GRAVITY=0.05,
    #     clip_coef=500,
    #     use_stable_dream_loss=False,
    # ),
    dict( #This one is also good!
        representation="raster bilateral",
        LEARNING_RATE=1e-0,
        GUIDANCE_SCALE=300,
        NUM_ITER=500,
        GRAVITY=0.05,
        # min_step=10,
        # max_step=600,
    ),
    dict(represntation="fourier",GRAVITY=0,NUM_ITER=300),
dict(representation="fourier"),

    # dict(representation="fourier"),
    # dict(),
    dict(representation="raster", LEARNING_RATE=1, GUIDANCE_SCALE=200,GRAVITY=.1),
    
    # dict(
    #     representation="raster bilateral",
    #     LEARNING_RATE=1e-0,
    #     GUIDANCE_SCALE=200,
    #     # NUM_ITER=500,
    #     GRAVITY=0.05,
    #     min_step=200,
    #     max_step=400,
    # ),
    # dict(
    #     representation="raster bilateral",
    #     LEARNING_RATE=1e-0,
    #     # NUM_ITER=500,
    #     GRAVITY=0.05,
    #     clip_coef=500,
    #     use_stable_dream_loss=True,
    #     GUIDANCE_SCALE=200,
    #     min_step=10,
    #     max_step=600,
    # ),
    # dict(NUM_ITER=500),
]


# experiment_setting_presets=[dict()]

things_to_try=[]

for prompts, urls in rp.shuffled(experiment_input_data):
    for prompt in prompts:
        for url in urls:
            things_to_try.append([prompt,url])

i=0
while True:
    # things_to_try=rp.shuffled(things_to_try)
    for prompt,url in things_to_try:
        
        for preset in experiment_setting_presets:
            i+=1
            print('EXPERIMENT NUMBER:',i)

            rp.ic(prompt,url,preset)

            results = run_peekaboo(
                prompt,
                url,
                **preset,
                output_folder_name='peekaboo_for_RPE'
            )

            get_result_thumbnail(results)
        from IPython.display import clear_output
        clear_output()

# OTHER STUFF

In [ ]:
results=run_peekaboo(
    # 'Green Luigi',
    # "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",


    # 'orange basketball',
    # 'brown american football',
    # 'black and white soccer ball on the bottom right',
    # 'https://m.media-amazon.com/images/I/711pmt372VL.jpg',
    
    'Beverly Crusher',
    # 'https://www.google.com/url?sa=i&url=https%3A%2F%2Fmemory-alpha.fandom.com%2Fwiki%2FBeverly_Crusher&psig=AOvVaw2_2z0ZVduCSQH1zpoNfPlY&ust=1675160497935000&source=images&cd=vfe&ved=0CA8QjRxqFwoTCNDkx6qJ7_wCFQAAAAAdAAAAABAE',
    'https://imgix.bustle.com/uploads/image/2020/6/30/5a0a61e2-5518-4bb9-b8ea-66e3b3986ad7-picard-cruhser.jpg?w=349&h=262&fit=max&auto=format%2Ccompress',
    
    # 'Harry Potter boy - daniel radcliffe - man with glasses',
    # 'Hermoine Granger - Emma Watson - Girl with long hair',
    # 'Hermoine',
     # "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
     # "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
    
    # 'Dory the fish',
    # 'https://akns-images.eonline.com/eol_images/Entire_Site/2016917/rs_600x600-161017135949-600.finding-dory-2.101716.jpg',

    # 'brass shiny saxophone on top',
     # "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",

    #Just CLIP
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.15, clip_coef=5000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.05, clip_coef=3000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=False, #BEST
    
    #Just DL
     # representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05, min_step=10, max_step=600,

    #Both
    representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=True, GUIDANCE_SCALE=200, min_step=10, max_step=600,
)

In [ ]:
results.output_folder

In [ ]:
#Mario vs Luigi Part 1
# peekaboo.s.min_step, peekaboo.s.max_step=200,600
results=run_peekaboo(
    # 'Green Luigi',
    # "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",


    # 'orange basketball',
    # 'brown american football',
    'black and white soccer ball on the bottom right',
    'https://m.media-amazon.com/images/I/711pmt372VL.jpg',
    
    # 'Harry Potter boy - daniel radcliffe - man with glasses',
    # 'Hermoine Granger - Emma Watson - Girl with long hair',
    # 'Hermoine',
     # "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
     # "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
    
    # 'Dory the fish',
    # 'https://akns-images.eonline.com/eol_images/Entire_Site/2016917/rs_600x600-161017135949-600.finding-dory-2.101716.jpg',

    # 'brass shiny saxophone on top',
     # "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",

    #Just CLIP
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.15, clip_coef=5000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.05, clip_coef=3000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=False, #BEST
    
    #Just DL
     # representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05, min_step=10, max_step=600,

    #Both
    representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=True, GUIDANCE_SCALE=200, min_step=10, max_step=600,
)

In [ ]:
peekaboo.s.min_step=200
peekaboo.s.max_step=600

#Mario vs Luigi Part 1
results=run_peekaboo(
    # 'Luigi', #200/600
    # 'Princess Peach', #200/600H, 
    # 'Mario',
    # 'Bowser from super smash bros; yellow shell; seen from the front', #NO; stable diffusion is too weak to draw this - or I have bad prompts
    
    
                     # "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                     # "http://images6.fanpop.com/image/photos/43400000/Mario-Peach-super-mario-bros-43431011-644-704.png",
                                         
                     # bilateral_kwargs=dict(kernel_size = 3,
                     #                  tolerance = .08,
                     #                  sigma = 5,
                     #                  iterations=2,
                     #                 ),
                     # representation='raster',
    
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
for _ in range(10):
    results=run_peekaboo('Luigi',
                         "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                         representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, GRAVITY=.05,
                         min_step=200,
                         max_step=400,
                         NUM_ITER=100,
                         )
    results_collection.append(results)



In [ ]:
#Mario vs Luigi Part 2
results=run_peekaboo('Luigi',
                     "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                     )

In [ ]:
#Mario vs Luigi Part 2
label = peekaboo.SimpleLabel('Luigi')
blend_factor=.5
label.embedding = label.embedding*(1+blend_factor) - peekaboo.SimpleLabel('Mario').embedding*blend_factor
results=run_peekaboo('Luigi',
                     "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                     label=label
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,400
#Wine and Cheese Part 1
results=run_peekaboo('Wine Glass half-full',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 1
results=run_peekaboo('Wine Glass half-full',
                     'Swiss Cheese',
                     'Grapes',
                     'Green Grapes',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,400
#Wine and Cheese Part 2
results=run_peekaboo('Swiss Cheese',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 2
results=run_peekaboo('Swiss Cheese',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
# https://americasrestaurant.com/wp-content/uploads/2022/04/what-do-bread-and-butter-pickles-taste-like.jpg
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 3
results=run_peekaboo(
    # 'piano keyboard',
    # 'violin',
    'brass shiny saxophone on top',
                     "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",
                     # representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
for _ in range(10):
    results=run_peekaboo(
        'Nemo Clownfish',
        'https://akns-images.eonline.com/eol_images/Entire_Site/2016917/rs_600x600-161017135949-600.finding-dory-2.101716.jpg',
        representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200,
        min_step=200, max_step=600,
        GRAVITY=.05,
        NUM_ITER=200,
    )
    results_collection.append(results)

In [ ]:
for _ in range(10):
    results=run_peekaboo(
        'brass shiny saxophone on top',
        "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",
        representation='raster bilateral',
        LEARNING_RATE=1e-0,
        GUIDANCE_SCALE=200,
        min_step=200,
        max_step=900,
        GRAVITY=.05,
        NUM_ITER=200,
    )
    results_collection.append(results)

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,400
#Wine and Cheese Part 3
results=run_ipeekaboo('grapes',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 3
results=run_peekaboo('Green grapes',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
#Harry Potter Part 1
results=run_peekaboo('Harry Potter boy - daniel radcliffe - man with glasses',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     NUM_ITER=300
                    )

In [ ]:
#Harry Potter Part 2
label = peekaboo.SimpleLabel('Hermoine Granger')
blend_factor=.5
label.embedding = label.embedding*(1+blend_factor) - peekaboo.SimpleLabel('Harry Potter').embedding*blend_factor
results=run_peekaboo('Hermoine',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     label=label
                     )


In [ ]:
#Harry Potter Part 2
label = peekaboo.SimpleLabel('Hermoine Granger - Emma Watson - Girl with long hair')
blend_factor=.5
label.embedding = label.embedding*(1+blend_factor) - peekaboo.SimpleLabel('Harry Potter boy - daniel radcliffe - man with glasses').embedding*blend_factor

results=run_peekaboo('Hermoine',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     label=label
                     )

In [ ]:
#Emma Watson
#Good in a row: |||
#Bad prompts: "Hermoine Granger aka Emma Watson"
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair',
                     "https://cometothedarkside97.files.wordpress.com/2021/05/images-2020-01-25t073939.551.jpeg?w=468",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Emma Watson
#Good in a row: 
#Bad prompts: 
results=run_peekaboo('Harry Potter boy - daniel radcliffe - man with glasses',
                     "https://prisiliaandayani.files.wordpress.com/2014/03/harry-ron-hermione-harry-potter-2422078-361-500.jpg",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Good in a row with fixed timestep 500: H,H,
#Bad prompts: 

# peekaboo.s.min_step=100
# peekaboo.s.max_step=500
for _ in range(10):
    results=run_peekaboo(
        'Harry Potter boy - daniel radcliffe - man with glasses',
        # 'Hermoine Granger - Emma Watson - Girl with long hair',

                         "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
                            # "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                         # "https://prisiliaandayani.files.wordpress.com/2014/03/harry-ron-hermione-harry-potter-2422078-361-500.jpg",

                         representation='raster bilateral',
                         # representation='raster',
                         # bilateral_kwargs=dict(kernel_size = 3,
                         #                  tolerance = .08,
                         #                  sigma = 5,
                         #                  iterations=2,
                         #                 ),
                         # representation='raster',
                         LEARNING_RATE=1e-0,
                         GUIDANCE_SCALE=200,
                         min_step=200,
                         max_step=400,
                         GRAVITY=.05,
                         NUM_ITER=200,
                         )
    results_collection.append(results)

In [ ]:
#Emma Watson
#Good in a row: ||X2X
#Good in a row with fixed timestep: |||||
#Bad prompts: 
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair',
                     "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
                     representation='raster bilateral',
                     # bilateral_kwargs=dict(kernel_size = 3,
                     #                  tolerance = .08,
                     #                  sigma = 5,
                     #                  iterations=2,
                     #                 ),
                     # representation='raster',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Emma Watson
#Good in a row: |||
#Bad prompts: "Emma Watson"
results=run_peekaboo('Hermoine Granger aka Emma Watson',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Harry Potter 
#Good in a row: ||
results=run_peekaboo('Daniel radcliffe',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Harry Potter 
#Good in a row: |||||
results=run_peekaboo('Harry Potter boy - daniel radcliffe - man with glasses',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Mario vs Luigi Part 1
#Good in a row: |||||
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Harry Potter Part 2
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair - left',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     NUM_ITER=300
                    )

In [ ]:
#Stable Diffusion Basic Demo
#This is a demo cell showing how to simply use Stable Diffusion to generate an image. This is usefull for prompt engineering.
rp.display_image(peekaboo.s.prompt_to_img('Cake'))

In [ ]:
#Stable Diffusion Basic Demo
#This is a demo cell showing how to simply use Stable Diffusion to generate an image. This is usefull for prompt engineering.
for _ in range(30):
    rp.display_image(peekaboo.s.prompt_to_img('Bowser from super smash bros; yellow shell; seen from the front'))

In [ ]:
rp.local_copy(results)

In [ ]:
results